In [5]:
!pip install dacon_submit_api-0.0.4-py3-none-any.whl

Processing ./dacon_submit_api-0.0.4-py3-none-any.whl


In [ ]:
#cd54fb1ab20010c1d742bd9789ca793fa6a83ebf47efb0818cbe576baf14f873

In [10]:
import pandas as pd
import numpy as np
import gc

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [36]:
import pandas as pd
import gc

class CARD_container():
    def __init__(self):
        self.data_splits = ["train", "test"]
        self.data_categories = {
            "회원정보": {"folder": "1.회원정보", "suffix": "회원정보", "var_prefix": "customer"},
            "신용정보": {"folder": "2.신용정보", "suffix": "신용정보", "var_prefix": "credit"},
            "승인매출정보": {"folder": "3.승인매출정보", "suffix": "승인매출정보", "var_prefix": "sales"},
            "청구정보": {"folder": "4.청구입금정보", "suffix": "청구정보", "var_prefix": "billing"},
            "잔액정보": {"folder": "5.잔액정보", "suffix": "잔액정보", "var_prefix": "balance"},
            "채널정보": {"folder": "6.채널정보", "suffix": "채널정보", "var_prefix": "channel"},
            "마케팅정보": {"folder": "7.마케팅정보", "suffix": "마케팅정보", "var_prefix": "marketing"},
            "성과정보": {"folder": "8.성과정보", "suffix": "성과정보", "var_prefix": "performance"}
        }
        self.months = ['07', '08', '09', '10', '11', '12']
        self.data = {}        # train/test 각각 month 단위로 저장
        self.train_data = {}  # train 통합 데이터프레임 저장
        self.test_data = {}   # test 통합 데이터프레임 저장
        

    def get_data(self, path='/home/cksgh8511/키움/card', months=['07'], data_splits=["train", "test"]):
        for split in data_splits:
            for category, info in self.data_categories.items():
                folder = info["folder"]
                suffix = info["suffix"]
                var_prefix = info["var_prefix"]

                for month in months:
                    file_path = f"{path}/{split}/{folder}/2018{month}_{split}_{suffix}.parquet"
                    variable_name = f"{var_prefix}_{split}_{month}"

                    try:
                        if(split=='train'):
                            self.train_data[variable_name] = pd.read_parquet(file_path)
                            print(f"{variable_name} is loaded from {file_path}")
                        elif(split=='test'):
                            self.test_data[variable_name] = pd.read_parquet(file_path)
                            print(f"{variable_name} is loaded from {file_path}")
                    except Exception as e:
                        print(f"Failed to load {file_path}: {e}")

        gc.collect()


In [37]:
card_container = CARD_container()
card_container.get_data(path='/home/cksgh8511/키움/card', months=['07'], data_splits=["train"])



customer_train_07 is loaded from /home/cksgh8511/키움/card/train/1.회원정보/201807_train_회원정보.parquet
credit_train_07 is loaded from /home/cksgh8511/키움/card/train/2.신용정보/201807_train_신용정보.parquet
sales_train_07 is loaded from /home/cksgh8511/키움/card/train/3.승인매출정보/201807_train_승인매출정보.parquet
billing_train_07 is loaded from /home/cksgh8511/키움/card/train/4.청구입금정보/201807_train_청구정보.parquet
balance_train_07 is loaded from /home/cksgh8511/키움/card/train/5.잔액정보/201807_train_잔액정보.parquet
channel_train_07 is loaded from /home/cksgh8511/키움/card/train/6.채널정보/201807_train_채널정보.parquet
marketing_train_07 is loaded from /home/cksgh8511/키움/card/train/7.마케팅정보/201807_train_마케팅정보.parquet
performance_train_07 is loaded from /home/cksgh8511/키움/card/train/8.성과정보/201807_train_성과정보.parquet


In [38]:
card_container.train_data

{'customer_train_07':           기준년월            ID  남녀구분코드     연령 Segment  회원여부_이용가능  회원여부_이용가능_CA  \
 0       201807  TRAIN_000000       2    40대       D          1             1   
 1       201807  TRAIN_000001       1    30대       E          1             1   
 2       201807  TRAIN_000002       1    30대       C          1             1   
 3       201807  TRAIN_000003       2    40대       D          1             1   
 4       201807  TRAIN_000004       2    40대       E          1             1   
 ...        ...           ...     ...    ...     ...        ...           ...   
 399995  201807  TRAIN_399995       2  70대이상       E          1             1   
 399996  201807  TRAIN_399996       2    50대       D          1             1   
 399997  201807  TRAIN_399997       1    30대       C          1             1   
 399998  201807  TRAIN_399998       1    40대       E          1             1   
 399999  201807  TRAIN_399999       2    40대       E          1             1   
 
     

In [71]:
import pandas as pd

class CARD_EDA():
    def __init__(self, card_container):
        self.card_container = card_container

    def get_columns(self, dfs='train'):
        if dfs == 'train':
            dfs = self.card_container.train_data
        elif dfs == 'test':
            dfs = self.card_container.test_data

        for name, df in dfs.items():
            print(f"\n[{name} 컬럼 목록] ({df.shape[1]} columns)")
            print(df.columns.tolist())

    def get_constant_feature(self, dfs='train', drop=True):
        if dfs == 'train':
            dfs = self.card_container.train_data
        elif dfs == 'test':
            dfs = self.card_container.test_data

        constant_features_dict = {}

        for name, df in dfs.items():
            constant_features = [
                col for col in df.columns
                if df[col].nunique(dropna=False) <= 1 and col != '기준년월'
            ]
            constant_features_dict[name] = constant_features
            if drop:
                df.drop(columns=constant_features, inplace=True)

            print(f"[{name}] 상수 feature 개수 (기준년월 제외): {len(constant_features)}개")
            if constant_features:
                print(" -", constant_features)
            else:
                print(" - 없음")

        self.constant_features_dict = constant_features_dict

    def get_date_features(self, dfs='train', drop=False):
        """ 날짜형 feature를 찾아 제거하는 메서드 """
        if dfs == 'train':
            dfs = self.card_container.train_data
        elif dfs == 'test':
            dfs = self.card_container.test_data

        date_features_dict = {}

        for name, df in dfs.items():
            date_features = [
                col for col in df.columns
                if (pd.api.types.is_datetime64_any_dtype(df[col]) or 'date' in col.lower() or '일자' in col)
            ]
            date_features_dict[name] = date_features
            if drop:
                df.drop(columns=date_features, inplace=True)

            print(f"[{name}] 날짜형 feature 개수: {len(date_features)}개")
            if date_features:
                print(" -", date_features)
            else:
                print(" - 없음")

        self.date_features_dict = date_features_dict

    def get_high_single_value_features(self, dfs='train', threshold=0.95, drop=True):
        """ 결측치 또는 단일값이 95% 이상인 feature 제거하는 메서드 """
        if dfs == 'train':
            dfs = self.card_container.train_data
        elif dfs == 'test':
            dfs = self.card_container.test_data

        high_single_value_features_dict = {}

        for name, df in dfs.items():
            high_single_features = []

            for col in df.columns:
                if df[col].isna().mean() >= threshold:
                    high_single_features.append(col)
                else:
                    top_freq = df[col].value_counts(normalize=True, dropna=False).values[0]
                    if top_freq >= threshold:
                        high_single_features.append(col)

            high_single_value_features_dict[name] = high_single_features

            if drop:
                df.drop(columns=high_single_features, inplace=True)

            print(f"[{name}] 95% 이상 결측/단일값 feature 개수: {len(high_single_features)}개")
            if high_single_features:
                print(" -", high_single_features)
            else:
                print(" - 없음")

        self.high_single_value_features_dict = high_single_value_features_dict


In [72]:
card_eda  = CARD_EDA(card_container)
card_eda.card_container

In [65]:
card_eda.get_columns(dfs = 'train')


[customer_train_07 컬럼 목록] (69 columns)
['기준년월', 'ID', '남녀구분코드', '연령', 'Segment', '회원여부_이용가능', '회원여부_이용가능_CA', '회원여부_이용가능_카드론', '소지여부_신용', '소지카드수_유효_신용', '소지카드수_이용가능_신용', '입회일자_신용', '입회경과개월수_신용', '회원여부_연체', '이용거절여부_카드론', '동의여부_한도증액안내', '수신거부여부_TM', '수신거부여부_DM', '수신거부여부_메일', '수신거부여부_SMS', '가입통신회사코드', '탈회횟수_누적', '최종탈회후경과월', '탈회횟수_발급6개월이내', '탈회횟수_발급1년이내', '거주시도명', '직장시도명', '마케팅동의여부', '유효카드수_신용체크', '유효카드수_신용', '유효카드수_신용_가족', '유효카드수_체크', '유효카드수_체크_가족', '이용가능카드수_신용체크', '이용가능카드수_신용', '이용가능카드수_신용_가족', '이용가능카드수_체크', '이용가능카드수_체크_가족', '이용카드수_신용체크', '이용카드수_신용', '이용카드수_신용_가족', '이용카드수_체크', '이용금액_R3M_신용체크', '이용금액_R3M_신용', '이용금액_R3M_신용_가족', '이용금액_R3M_체크', '_1순위카드이용금액', '_1순위카드이용건수', '_1순위신용체크구분', '_2순위카드이용금액', '_2순위카드이용건수', '_2순위신용체크구분', '최종유효년월_신용_이용가능', '최종유효년월_신용_이용', '최종카드발급일자', '보유여부_해외겸용_본인', '이용가능여부_해외겸용_본인', '이용여부_3M_해외겸용_본인', '보유여부_해외겸용_신용_본인', '이용가능여부_해외겸용_신용_본인', '이용여부_3M_해외겸용_신용_본인', '연회비발생카드수_B0M', '기본연회비_B0M', '제휴연회비_B0M', '청구금액_기본연회비_B0M', '청구금액_제휴연회비_B0M', '카드신청건수', 'Life_Stage', '최종카드

In [66]:
card_eda.get_constant_feature(drop = True)

[customer_train_07] 상수 feature 개수 (기준년월 제외): 0개
 - 없음
[credit_train_07] 상수 feature 개수 (기준년월 제외): 0개
 - 없음
[sales_train_07] 상수 feature 개수 (기준년월 제외): 0개
 - 없음
[billing_train_07] 상수 feature 개수 (기준년월 제외): 0개
 - 없음
[balance_train_07] 상수 feature 개수 (기준년월 제외): 0개
 - 없음
[channel_train_07] 상수 feature 개수 (기준년월 제외): 0개
 - 없음
[marketing_train_07] 상수 feature 개수 (기준년월 제외): 0개
 - 없음
[performance_train_07] 상수 feature 개수 (기준년월 제외): 0개
 - 없음


In [69]:
card_eda.get_constant_feature(dfs='train', drop=False)

[customer_train_07] 상수 feature 개수 (기준년월 제외): 0개
 - 없음
[credit_train_07] 상수 feature 개수 (기준년월 제외): 0개
 - 없음
[sales_train_07] 상수 feature 개수 (기준년월 제외): 0개
 - 없음
[billing_train_07] 상수 feature 개수 (기준년월 제외): 0개
 - 없음
[balance_train_07] 상수 feature 개수 (기준년월 제외): 0개
 - 없음
[channel_train_07] 상수 feature 개수 (기준년월 제외): 0개
 - 없음
[marketing_train_07] 상수 feature 개수 (기준년월 제외): 0개
 - 없음
[performance_train_07] 상수 feature 개수 (기준년월 제외): 0개
 - 없음


In [73]:
card_eda.get_date_features(dfs='train', drop=False)

[customer_train_07] 날짜형 feature 개수: 2개
 - ['입회일자_신용', '최종카드발급일자']
[credit_train_07] 날짜형 feature 개수: 1개
 - ['RV신청일자']
[sales_train_07] 날짜형 feature 개수: 8개
 - ['최종이용일자_기본', '최종이용일자_신판', '최종이용일자_CA', '최종이용일자_카드론', '최종이용일자_체크', '최종이용일자_일시불', '최종이용일자_할부', '최종카드론_대출일자']
[billing_train_07] 날짜형 feature 개수: 0개
 - 없음
[balance_train_07] 날짜형 feature 개수: 1개
 - ['연체일자_B0M']
[channel_train_07] 날짜형 feature 개수: 0개
 - 없음
[marketing_train_07] 날짜형 feature 개수: 0개
 - 없음
[performance_train_07] 날짜형 feature 개수: 0개
 - 없음


In [74]:
card_eda.get_high_single_value_features(dfs='train', threshold=0.95, drop=False)

[customer_train_07] 95% 이상 결측/단일값 feature 개수: 17개
 - ['기준년월', '회원여부_이용가능', '소지여부_신용', '회원여부_연체', '탈회횟수_발급6개월이내', '유효카드수_신용_가족', '유효카드수_체크_가족', '이용가능카드수_신용_가족', '이용가능카드수_체크_가족', '이용카드수_신용_가족', '이용금액_R3M_신용_가족', '_1순위신용체크구분', '연회비발생카드수_B0M', '기본연회비_B0M', '제휴연회비_B0M', '청구금액_기본연회비_B0M', '청구금액_제휴연회비_B0M']
[credit_train_07] 95% 이상 결측/단일값 feature 개수: 13개
 - ['기준년월', '자발한도감액횟수_R12M', '자발한도감액금액_R12M', '자발한도감액후경과월', '특별한도보유여부_R3M', '연체감액여부_R3M', '한도심사요청건수', '한도요청거절건수', '한도심사요청후경과월', '한도심사거절후경과월', '시장단기연체여부_R6M', '시장단기연체여부_R3M', '시장연체상환여부_R6M']
[sales_train_07] 95% 이상 결측/단일값 feature 개수: 106개
 - ['기준년월', '이용건수_할부_유이자_B0M', '이용건수_CA_B0M', '이용건수_카드론_B0M', '이용금액_할부_유이자_B0M', '이용금액_CA_B0M', '이용금액_카드론_B0M', '이용후경과월_부분무이자', '이용건수_부분무이자_R12M', '이용금액_부분무이자_R12M', '최대이용금액_부분무이자_R12M', '이용개월수_부분무이자_R12M', '이용건수_부분무이자_R6M', '이용건수_카드론_R6M', '이용금액_부분무이자_R6M', '이용금액_카드론_R6M', '이용개월수_부분무이자_R6M', '이용개월수_카드론_R6M', '이용건수_부분무이자_R3M', '이용건수_카드론_R3M', '이용금액_카드론_R3M', '이용개월수_부분무이자_R3M', '이용개월수_카드론_R3M', '교통_통행료이용금액', '

In [90]:
def get_unique_info_for_df(series):
    value_counts = series.value_counts(dropna=False)
    return pd.Series({
        'n_unique': series.nunique(dropna=False),
        'value_counts': value_counts.to_dict()
    })

result_df = card_eda.card_container.train_data['customer_train_07'][
    ['기준년월', '회원여부_이용가능', '소지여부_신용', '회원여부_연체', '탈회횟수_발급6개월이내', 
     '유효카드수_신용_가족', '유효카드수_체크_가족', '이용가능카드수_신용_가족', '이용가능카드수_체크_가족',
     '이용카드수_신용_가족', '이용금액_R3M_신용_가족', '_1순위신용체크구분', 
     '연회비발생카드수_B0M', '기본연회비_B0M', '제휴연회비_B0M', 
     '청구금액_기본연회비_B0M', '청구금액_제휴연회비_B0M']
].apply(get_unique_info_for_df).T

In [91]:
result_df.to_excel()

,n_unique,value_counts
기준년월,1,{201807: 400000}
회원여부_이용가능,2,"{1: 382820, 0: 17180}"
소지여부_신용,2,"{1: 397001, 0: 2999}"
회원여부_연체,2,"{0: 391687, 1: 8313}"
탈회횟수_발급6개월이내,2,"{0: 384478, 1: 15522}"
유효카드수_신용_가족,3,"{0: 388738, 1: 8187, 2: 3075}"
유효카드수_체크_가족,2,"{0: 399993, 1: 7}"
이용가능카드수_신용_가족,3,"{0: 388804, 1: 8137, 2: 3059}"
이용가능카드수_체크_가족,2,"{0: 399997, 1: 3}"
이용카드수_신용_가족,3,"{0: 389769, 1: 7413, 2: 2818}"
